# Training eines Transfer Learning Models

In diesem Jupyter Notebook wird ein Transfer Learning Model trainiert.
Alle Transfer Learning Models wurden mit diesem Skript trainiert und nur jeweils Pfade und Parameter angepasst.

Das Training wurde auf Google Colab durchgeführt, daher können Pfadanpassungen für die Ausführung notwendig sein.

Importe und Konstanten

In [ ]:
# Imports needed
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

img_height = 320
img_width = 180

batch_size = 15
name = "one_category"


nClasses_1 = 355

input_shape = (img_height, img_width, 3) #3 for rgb, 1 or grayscale
SRC_DIR_1 = "/content/training/" + name

Hierdurch wird eine Verbindung zu Google Drive hergestellt, um die Trainingsdaten zu erreichen. 
Diese müssen für jede Sitzung hochgeladen werden. Der Datensatz wird aus Google Drive geladen und in das lokale Filesystem von Colab entpackt.

Zudem werden die Ordner angelegt, in die die Modelle später gespeichert werden.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!mkdir /content/gdrive/MyDrive/Thesis_ColabNotebooks/models/TransferLearning
!mkdir /content/gdrive/MyDrive/Thesis_ColabNotebooks/models/TransferLearning/tf

In [ ]:
!unzip /content/gdrive/MyDrive/Thesis_ColabNotebooks/TrainData/one_category -d /content/training

Im nächsten Abschnitt wird eine GPU auf Colab gesucht. Damit kann ein Model auf der GPU trainiert werden statt auf der CPU.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Preprocess nimmt die Bilder und splittet sie in Trainins- und Testdaten. Dabei werden die Bilder jeweils gelabelt und zwar abhängig des Ordners, in dem sie sich befinden. Ein Bild in Ordner A0012 bekommt dementsprechend das Label A0012. Das Label steht für Gebäudeteil A, Stockwerk 0, an Position 01 im Gang und Blickrichtung 2. 

In [ ]:
def preprocess(source_dir):
    ds_train = tf.keras.preprocessing.image_dataset_from_directory(
        source_dir,
        labels="inferred",
        label_mode="categorical",  #int categorical, binary
        color_mode="rgb",
        batch_size=batch_size,
        image_size=(img_height, img_width),  # reshape if not in this size
        shuffle=True,
        seed=123,
        validation_split=0.1,
        subset="training",
    )

    ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
        source_dir,
        labels="inferred",
        label_mode="categorical",  # categorical, binary
        color_mode="rgb",
        batch_size=batch_size,
        image_size=(img_height, img_width),  # reshape if not in this size
        shuffle=True,
        seed=123,
        validation_split=0.1,
        subset="validation",
    )

    return (ds_train, ds_validation)

Vorbereitung der Trainingsdaten

In [ ]:
(ds_train_m1, ds_validation_m1) = preprocess(SRC_DIR_1)

In diesem Abschnitt wird das Basismodel (MobileNetV2) geladen und der Output Layer durch einen GlobalAveragePooling2D Layer, einen Dropout Layer und einen Dense Layer ersetzt.

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape = input_shape, include_top = False, weights = "imagenet")
base_model.trainable = False
model = tf.keras.Sequential([base_model,
                                 tf.keras.layers.GlobalAveragePooling2D(),
                                 tf.keras.layers.Dropout(0.2),
                                 tf.keras.layers.Dense(nClasses_1, activation="softmax")                                     
                                ])

Nachdem der Output Layer ersetzt wurde, kann das Model kompiliert, trainiert werden und gespeichert werden

In [ ]:
base_learning_rate = 0.00001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(ds_train_m1,epochs = 175 , validation_data = ds_validation_m1)
model.save('/content/gdrive/MyDrive/Thesis_ColabNotebooks/models/TransferLearning/' + name )